#### Post process data for data viz 

In [65]:
import os,sys
sys.path.insert(0,'../../libs')
import pandas as pd

In [66]:
data_folder = '/data/chuang/Language_Model_Training_Data/Models/Topic_Models/step_10000'
chats_out_folder = os.path.join(data_folder,'figs')
#merged_out_path = os.path.join(data_folder,'topic_v2_merged_agg_data.csv')
#merged_out_simple_path = os.path.join(data_folder,'topic_v2_merged_agg_data_simple.csv')
merged_out_simple_path = os.path.join(data_folder,'AIV_topic_LLM_Eval_simple.csv')


In [67]:
df = pd.read_csv(merged_out_simple_path)

#### Temp merge some topics for charting purposes 

In [76]:
df.loc[df['level_0'] == 'Statistical Analysis', 'level_0'] = 'Statistics'

In [80]:
# Define the mapping
mapping = {
    'Financial Inclusion': 'Gender',
    'Sustainable Growth': 'Structural Reform',
    'Infrastructure': 'Structural Reform'
}

# Apply the mapping where 'level_0' is "Structural/Social"
df.loc[df['level_0'] == 'Structural/Social', 'level_1'] = df['level_1'].map(mapping).fillna(df['level_1'])

#### 

pip install git+https://github.com/JosephBARBIERDARNAL/pypalettes.git
- https://python-graph-gallery.com/590-advanced-treemap/
- other [examples](https://python-graph-gallery.com/web-stacked-area-with-inflexion-arrows/)

In [68]:
# load libraries
import squarify # pip install squarify (algorithm for treemap)
import matplotlib.pyplot as plt
#from pypalettes import load_cmap
import pandas as pd
from highlight_text import fig_text
import numpy as np
import plotly.express as px
import pandas as pd


- Treemap 

In [82]:
collapsed_df = df.groupby(['level_0', 'level_1'], as_index=False)['File_Name'].count()
collapsed_df.head(2)

,level_0,level_1,File_Name
0,External,Balance Of Payments,16288
1,External,Commodity Market,3648


In [83]:
def treemap_by_topic(collapsed_df,title="Treemap of AIV Topic Distribution"):
    # Create the treemap
    fig = px.treemap(collapsed_df, 
                    path=['level_0', 'level_1'], 
                    values='File_Name',
                    #color='Name',
                    #color_continuous_scale='RdBu',
                    width=1000, 
                    height=800,
                    title=title)
    fig.update_traces(hovertemplate='labels=%{label}<br>total_count=%{value}<extra></extra>')
    return fig

In [84]:
fig = treemap_by_topic(collapsed_df,title="Treemap of AIV Topic Distribution")
fig.write_image(os.path.join(chats_out_folder,"Treemap_AIV_Topic_Distribution.png"))
fig.write_html(os.path.join(chats_out_folder,"Treemap_AIV_Topic_Distribution.html"))
# Show the plot
fig.show()

- by income and region 

In [38]:
# for G in ['income','REO Region']:
#     collapsed_df = df.groupby(['Year','level_0','level_1',G], as_index=False)['File_Name'].count()
#     for country_group in collapsed_df[G].unique():
#         sub_collapsed_df = collapsed_df[collapsed_df[G] == country_group]
#         fig = treemap_by_topic(sub_collapsed_df,title='{} Countries Treemap of AIV Topic Distribution'.format(country_group))
#         #fig.show()

- overall distribution timeseries 

In [85]:
collapsed_df = df.groupby(['Year','level_0'], as_index=False)['File_Name'].count()
collapsed_df.head(2)

,Year,level_0,File_Name
0,2008,External,2458
1,2008,Financial,1837


In [86]:
def time_series_by_topic(collapsed_df,title='Topic Over Time'):
    fig = px.area(collapsed_df, 
                x="Year", 
                y="File_Name", 
                color="level_0", 
                line_group="level_0",
                height=500,
                width=1000,
                title=title,
                )
    fig.update_layout(legend=dict(
                                title=dict(
                                    text='High Level Topics',
                                    font=dict(size=14)  # Optional: set the font size
                                ),),
                        yaxis_title='# of paragraphs'
        
                        )
    return fig


In [87]:
fig = time_series_by_topic(collapsed_df)
fig.write_image(os.path.join(chats_out_folder,"TimeSeries_AIV_Topic_counts.png"))
fig.write_html(os.path.join(chats_out_folder,"TimeSeries_AIV_Topic_counts.html"))
fig.show()

- Timeseries by income and regions 

In [88]:
collapsed_df = df.groupby(['Year','level_0','income'], as_index=False)['File_Name'].count()

In [41]:
# for country_group in collapsed_df['income'].unique():
#     income_collapsed_df = collapsed_df[collapsed_df['income'] == country_group]
#     fig = time_series_by_topic(income_collapsed_df,title='{} Countries Topic Over Time'.format(country_group))
#     #fig.show()

- Time Series in percentage

In [89]:
collapsed_df = df.groupby(['Year','level_0'], as_index=False)['File_Name'].count()
collapsed_df['File_Name'] = collapsed_df.groupby('Year')['File_Name'].transform(lambda x: (x / x.sum()*100))
collapsed_df.head(2)

,Year,level_0,File_Name
0,2008,External,18.381693
1,2008,Financial,13.737661


In [90]:
def time_series_by_topic_percentage(collapsed_df,title='Topic Distribution Over Time'):
    fig = px.area(collapsed_df, 
                x="Year", 
                y="File_Name", 
                color="level_0", 
                line_group="level_0",
                height=500,
                width=1000,
                title=title,
                )
    fig.update_layout(legend=dict(
                                title=dict(
                                    text='High Level Topics',
                                    font=dict(size=14)  # Optional: set the font size
                                ),),
                            yaxis=dict(
                                        title='Percentage',  # Optional: Set the y-axis title
                                        range=[0, 100]  # Set the y-axis range from 0 to 100
                                    ),
        
                        )
    return fig

fig = time_series_by_topic_percentage(collapsed_df)
fig.write_image(os.path.join(chats_out_folder,"TimeSeries_Topic_Distribution.png"))
fig.write_html(os.path.join(chats_out_folder,"TimeSeries_Topic_Distribution.html"))
fig.show()

- generate charts by country group

In [48]:
# for G in ['income','REO Region']:
#     collapsed_df = df.groupby(['Year','level_0',G], as_index=False)['File_Name'].count()
    
#     for country_group in collapsed_df[G].unique():
#         sub_collapsed_df = collapsed_df[collapsed_df[G] == country_group].copy()
#         sub_collapsed_df['File_Name'] = sub_collapsed_df.groupby('Year')['File_Name'].transform(lambda x: (x / x.sum()*100))
#         fig = time_series_by_topic_percentage(sub_collapsed_df,title='{} Countries Topic Distribution Over Time'.format(country_group))
#         fig.write_image(os.path.join(chats_out_folder,"Topic_Distribution_Over_Time_{}.png".format(country_group)))
#         #fig.write_html(os.path.join(chats_out_folder,"TimeSeries_Topic_Distribution.html"))
#         #fig.show()

- Show trends in structural topics 

In [92]:
collapsed_df = df.groupby(['Year','level_0','level_1'], as_index=False)['File_Name'].count()
collapsed_df = collapsed_df[collapsed_df['level_0']=='Structural/Social']
collapsed_df.head(2)

,Year,level_0,level_1,File_Name
65,2008,Structural/Social,Business Environment,221
66,2008,Structural/Social,Climate,29


In [93]:
def time_series_level1(collapsed_df,title='Structural/Social Topic Over Time'):
    fig = px.area(collapsed_df, 
                x="Year", 
                y="File_Name", 
                color="level_1", 
                line_group="level_1",
                height=500,
                width=1000,
                title=title,
                )
    fig.update_layout(legend=dict(
                                title=dict(
                                    text='High Level Topics',
                                    font=dict(size=14)  # Optional: set the font size
                                ),),
                        yaxis_title='# of paragraphs'
        
                        )
    return fig

fig = time_series_level1(collapsed_df,title='Structural/Social Topic Over Time')
fig.write_image(os.path.join(chats_out_folder,"TimeSeries_AIV_Topic_counts_Structural.png"))
fig.write_html(os.path.join(chats_out_folder,"TimeSeries_AIV_Topic_counts_Structural.html"))
fig.show()

- create charts by country groups 

In [94]:
# for G in ['income','REO Region']:
#     #collapsed_df = df.groupby(['Year','level_0',G], as_index=False)['File_Name'].count()
#     collapsed_df = df.groupby(['Year','level_0','level_1',G], as_index=False)['File_Name'].count()
#     collapsed_df = collapsed_df[collapsed_df['level_0']=='Structural/Social']
#     for country_group in collapsed_df[G].unique():
#         sub_collapsed_df = collapsed_df[collapsed_df[G] == country_group].copy()
#         #sub_collapsed_df['File_Name'] = sub_collapsed_df.groupby('Year')['File_Name'].transform(lambda x: (x / x.sum()*100))
#         fig = time_series_level1(sub_collapsed_df,title='{} Countries Structural Topic Counts Over Time'.format(country_group))
#         fig.write_image(os.path.join(chats_out_folder,"TimeSeries_AIV_Topic_counts_Structural_{}.png".format(country_group)))
#         fig.write_html(os.path.join(chats_out_folder,"TimeSeries_AIV_Topic_counts_Structural_{}.html".format(country_group)))
#         #fig.show()

- show structural as percentages 

In [95]:
collapsed_df = df.groupby(['Year','level_0','level_1'], as_index=False)['File_Name'].count()
collapsed_df = collapsed_df[collapsed_df['level_0']=='Structural/Social']
collapsed_df['File_Name'] = collapsed_df.groupby('Year')['File_Name'].transform(lambda x: (x / x.sum()*100))

In [96]:
def time_series_level_1_percentage(collapsed_df, title = 'Structural/Social Topic Distribution Over Time'):
    fig = px.area(collapsed_df, 
                x="Year", 
                y="File_Name", 
                color="level_1", 
                line_group="level_1",
                height=500,
                width=1000,
                title=title,
                )
    fig.update_layout(legend=dict(
                                title=dict(
                                    text='High Level Topics',
                                    font=dict(size=14)  # Optional: set the font size
                                ),),
                            yaxis=dict(
                                        title='Percentage',  # Optional: Set the y-axis title
                                        range=[0, 100]  # Set the y-axis range from 0 to 100
                                    ),
        
                        )
    return fig
fig = time_series_level_1_percentage(collapsed_df)
fig.write_image(os.path.join(chats_out_folder,"TimeSeries_AIV_Topic_Distribution_Structural.png"))
fig.write_html(os.path.join(chats_out_folder,"TimeSeries_AIV_Topic_Distribution_Structural.html"))
fig.show()

In [64]:
# for G in ['income','REO Region']:
#     #collapsed_df = df.groupby(['Year','level_0',G], as_index=False)['File_Name'].count()
#     collapsed_df = df.groupby(['Year','level_0','level_1',G], as_index=False)['File_Name'].count()
#     collapsed_df = collapsed_df[collapsed_df['level_0']=='Structural/Social']
#     for country_group in collapsed_df[G].unique():
#         sub_collapsed_df = collapsed_df[collapsed_df[G] == country_group].copy()
#         sub_collapsed_df['File_Name'] = sub_collapsed_df.groupby('Year')['File_Name'].transform(lambda x: (x / x.sum()*100))
#         fig = time_series_level_1_percentage(sub_collapsed_df,title='{} Countries Structural Topic Distribution Over Time'.format(country_group))
#         fig.write_image(os.path.join(chats_out_folder,"TimeSeries_AIV_Topic_Distribution_Structural_{}.png".format(country_group)))
#         fig.write_html(os.path.join(chats_out_folder,"TimeSeries_AIV_Topic_Distribution_Structural_{}.html".format(country_group)))
#         #fig.show()

In [97]:
df.columns

Index(['index', 'File_Name', 'Title', 'Country Code', 'Country_Name', 'Year',
       'income', 'REO Region', 'par', 'final_topic_id_llm_revised',
       'CustomName', 'level_0', 'level_1', 'level_2'],
      dtype='object')

Look at structural sub categories as percent of total , by income

In [146]:
def collapse_by_country_group(df,c_group='income',level_0_filter='Structural/Social'):
    collapsed_df = df.groupby(['Year','level_0','level_1',c_group], as_index=False)['File_Name'].count()
    collapsed_df['File_Name'] = collapsed_df.groupby(['Year','income'])['File_Name'].transform(lambda x: (x / x.sum()*100))
    ## just to verify the calculation is correct 
    print(collapsed_df[(collapsed_df['Year'] == 2008) & (collapsed_df['income'] == 'AE')]['File_Name'].sum())
    
    if level_0_filter:
        ## keek only Structural topic
        collapsed_df = collapsed_df[collapsed_df['level_0']==level_0_filter]
    
    ## only compare 2 years
    compare_df = collapsed_df[collapsed_df['Year'].isin([2018, 2023])]
    compare_df['Year'] = compare_df['Year'].astype(str)

    return compare_df

In [147]:
compare_df = collapse_by_country_group(df,c_group='income',level_0_filter='Structural/Social')

100.0


/tmp/ipykernel_222866/380428797.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [148]:

fig = px.bar(compare_df, x="Year", y="File_Name",
             facet_col="income",color="level_1",
            height=500,width=1000)
fig.update_layout(legend=dict(
                            title=dict(
                                text='Structural/Social Topics',
                                font=dict(size=14)  # Optional: set the font size
                            ),),
                        yaxis=dict(
                                    title='Percentage',  # Optional: Set the y-axis title
                                    #range=[0, 100]  # Set the y-axis range from 0 to 100
                                ),
    
                    )
fig.show()

In [149]:
compare_df = collapse_by_country_group(df,c_group='REO Region',level_0_filter='Structural/Social')

KeyError: 'income'

In [150]:

fig = px.bar(compare_df, x="Year", y="File_Name",
             facet_col="REO Region",color="level_1",
            height=500,width=1000)
fig.update_layout(legend=dict(
                            title=dict(
                                text='Structural/Social Topics',
                                font=dict(size=14)  # Optional: set the font size
                            ),),
                        yaxis=dict(
                                    title='Percentage',  # Optional: Set the y-axis title
                                    #range=[0, 100]  # Set the y-axis range from 0 to 100
                                ),
    
                    )
fig.show()

ValueError: Value of 'facet_col' is not the name of a column in 'data_frame'. Expected one of ['Year', 'level_0', 'level_1', 'income', 'File_Name'] but received: REO Region